In [ ]:
from collections import Counter
import numpy as np
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

## Building a Decision Tree from Scratch

### **Node Representation**

- **Purpose**: Represents a node in the decision tree, which can either be a leaf or an internal node.
- **Attributes**:
  - `is_leaf` (bool): To determine if the node is a leaf node.
  - `feature_index` (int): The feature index used to split.
  - `threshold` (float): The threshold value used to split the data.
  - `prediction` (int): The value of the prediction (if the node is a leaf).
  - `left` (dict): Left child node.
  - `right` (dict): Right child node.

In [ ]:
def create_node(
    is_leaf: bool,
    feature_index: int = -1,
    threshold: float = -1,
    prediction: float = None,
) -> dict:
    """
    Creates a decision tree node.

    Args:
        is_leaf (bool): Whether the node is a leaf node.
        feature_index (int): The feature index used for splitting.
        threshold (float): The threshold value for splitting.
        prediction (float): The prediction value for leaf nodes.

    Returns:
        dict: A dictionary representing a decision tree node.
    """

    node = {
        "is_leaf": is_leaf,
        "feature_index": feature_index,
        "threshold": threshold,
        "prediction": prediction,
        "left": None, # set to none initially. can assign child nodes after init
        "right": None
    }
    return node

### **Splitting Dataset**

- **Purpose**: Split the dataset into left and right parts based on a given feature and threshold.
  - Iterate through each instance in the dataset.
  - Compare the value of the selected feature with the given threshold.
  - Append the instance to either the left or right split based on whether the feature value is less than or equal to the threshold.
  - Make sure both the features and labels are split according

In [ ]:
def split_dataset(
    df: pd.DataFrame, feature_index: str, threshold: float
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Splits the dataset based on a feature and a threshold.

    Args:
        df (pd.DataFrame): The dataset including features and labels.
        feature_index (str): The feature to split on.
        threshold (float): The threshold value to use for the split.

    Returns:
        tuple: A tuple containing the left and right datasets.
    """

    column_list  = df.columns.tolist()
    column_index = column_list.index(feature_index)
    left_split  = []
    right_split = []
    numpy_dataset = df.to_numpy() # converting df to a numpy array for efficiency

    for instance in numpy_dataset:
      if instance[column_index] <= threshold:
        left_split.append(instance)
      else:
        right_split.append(instance)

    left_split = pd.DataFrame(left_split, columns=column_list)
    right_split = pd.DataFrame(right_split, columns=column_list)

    return left_split, right_split

### **Checking Purity**

- **Purpose**: Determine if all the labels are the same.
  - If all labels are identical, the node should be considered pure, and it will become a leaf node.
  - Check if all elements are the same.
  - This function helps determine whether further splitting is necessary.

In [ ]:
def is_pure(labels: pd.Series) -> bool:
    """
    Checks if all the labels are the same.

    Args:
        labels (pd.Series): The label values.

    Returns:
        bool: True if all labels are the same, False otherwise.
    """

    if labels.nunique() == 1:
      return True
    else:
      return False

### **Finding the Best Split**

- **Purpose**: Find the best feature and threshold to split the dataset.
  - Iterate through each feature and determine possible thresholds.
  - Use entropy as the metric to evaluate the quality of the split.
  - For each threshold, calculate the entropy of the resulting splits and choose the one with the lowest entropy.
  - The best split will minimize the entropy, meaning the resulting splits are as pure as possible.


In [ ]:
def calculate_thresholds(column: np.array) -> list[float]:
  thresholds = []
  sorted_column = np.sort(column)
  for i in range(len(sorted_column) - 1): # -1 because there are n-1 thresholds
    thresholds.append((sorted_column[i] + sorted_column[i+1]) / 2)
  return thresholds

def find_best_split(
    df: pd.DataFrame, features: list[str], label_column: str
) -> tuple[str, float]:
    """
    Finds the best feature and threshold to split the dataset by minimizing entropy.

    Args:
        df (pd.DataFrame): The dataset including features and labels.
        features (list[str]): The list of feature column names.
        label_column (str): The label column name.

    Returns:
        tuple: The feature name and threshold value for the best split.
    """

    best_entropy = float("inf")
    best_feature = ""
    best_threshold = -1

    # making each column a numpy array for efficiency
    numpy_columns = []
    for feature in features:
      numpy_columns.append(df[feature].to_numpy())

    for i in range(len(features)): # for every feature
      possible_thresholds = calculate_thresholds(numpy_columns[i])
      for threshold in possible_thresholds: # go over every threshold
        left_split, right_split = split_dataset(df, features[i], threshold)
        weighted_entropy = calculate_entropy(left_split, right_split)
        if weighted_entropy < best_entropy: # pick the best threshold based on smallest entropy
          best_entropy = weighted_entropy
          best_feature = features[i]
          best_threshold = threshold

    return best_feature, best_threshold

### **Calculating Entropy**

- **Purpose**: Calculate the entropy for a given split of labels.
  - Entropy is a measure of impurity, where lower values indicate a purer split.
  - For each subset of labels (left and right), calculate the proportion of each class.
  - Use the formula for entropy
  - The entropy for the split is the weighted average of the left and right entropies.


In [ ]:
def calculate_entropy(left_labels: pd.Series, right_labels: pd.Series) -> float:
    """
    Calculates the entropy for a split.

    Args:
        left_labels (pd.Series): The label values for the left split.
        right_labels (pd.Series): The label values for the right split.

    Returns:
        float: The entropy value for the split.
    """
    left_size = len(left_labels)
    right_size = len(right_labels)
    total_size = left_size + right_size

    if total_size == 0:
        return 0

    def entropy(labels):
      p = {}
      labels = labels.to_numpy().flatten()
      for label in labels:
        if label in p:
          p[label] += 1
        else:
          p[label] = 1
      for key in p:
        p[key] = p[key] / len(labels)

      summation = 0
      for prob in p.values():
        pi_log_pi = prob * np.log2(prob)
        summation += pi_log_pi
      return -summation

    # Calculate entropy for both left and right splits
    entropy_left = entropy(left_labels)
    entropy_right = entropy(right_labels)

    # Weighted average of both entropies
    weighted_average = (left_size / total_size) * entropy_left + (right_size / total_size) * entropy_right
    return weighted_average

### **Calculating Prediction**

- **Purpose**: Calculate the prediction value for a leaf node (most common label).
- **Description**:
  - This function returns the most common label in the dataset, which is used as the prediction for a leaf node.


In [ ]:
def calculate_prediction(labels: pd.Series) -> int:
    """
    Calculates the prediction value for a leaf node.

    Args:
        labels (pd.Series): The label values.

    Returns:
        int: The most common value of the labels.
    """

    return labels.value_counts().index[0]

### **Building the Decision Tree**

- **Purpose**: Recursively build the decision tree by finding the best split and creating nodes.
- **Steps**:
  - Start with the current depth as 0 and increase it with each recursive call.
  - Stop the recursion if the current depth reaches the maximum depth or if the node is pure.
  - Use the `find_best_split` function to determine the best feature and threshold for splitting.
  - Create left and right child nodes recursively and attach them to the current node.

In [ ]:
def build_tree(
    df: pd.DataFrame,
    features: list[str],
    label_column: str,
    current_depth: int,
    max_depth: int,
) -> dict:
    """
    Builds the decision tree recursively.

    Args:
        df (pd.DataFrame): The dataset including features and labels.
        features (list[str]): The list of feature column names.
        label_column (str): The label column name.
        current_depth (int): The current depth of the tree.
        max_depth (int): The maximum allowed depth of the tree.

    Returns:
        dict: The root node of the decision tree.
    """
    # base case
    if current_depth >= max_depth or is_pure(df[label_column]):
      return create_node(
          is_leaf = True,
          prediction = calculate_prediction(df[label_column])
      )

    # find best feature
    best_feature, best_threshold = find_best_split(df, features, label_column)
    node = create_node(
        is_leaf = False,
        feature_index = features.index(best_feature),
        threshold = best_threshold
    )

    # split the data
    left_split, right_split = split_dataset(df, best_feature, best_threshold)
    node["left"] = build_tree(left_split, features, label_column, current_depth+1, max_depth)
    node["right"] = build_tree(right_split, features, label_column, current_depth+1, max_depth)

    return node

### **Predicting Using the Tree**

- **Purpose**: Predict the label for a given feature vector using the trained decision tree.
- **Steps**:
  - Start at the root of the tree and traverse down to a leaf node.
  - For each internal node, decide whether to move left or right based on the feature value and threshold.
  - When a leaf node is reached, return its prediction value.

In [ ]:
def predict(tree: dict, sample: pd.Series) -> int:
    """
    Predicts the label for a given feature vector using the decision tree.

    Args:
        tree (dict): The root node of the decision tree.
        sample (pd.Series): The feature vector.

    Returns:
        int: The predicted label.
    """

    if tree["is_leaf"] == True:
      return tree["prediction"]

    elif sample[tree["feature_index"]] <= tree["threshold"]:
      return predict(tree["left"], sample)

    else:
      return predict(tree["right"], sample)

### Using a Real-World Dataset and Comparison with Scikit-Learn

In [ ]:
# Load the Iris dataset
iris = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data", header=None)
iris.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']

In [ ]:
# Prepare features and labels
features = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
label_column = 'class'
iris['class'] = iris['class'].apply(lambda x: 0 if x == 'Iris-setosa' else (1 if x == 'Iris-versicolor' else 2))

In [ ]:
# Split the dataset into training and testing
X_train, X_test = train_test_split(iris, test_size=0.2, random_state=42)

In [ ]:
max_depth = 3
decision_tree = build_tree(X_train, features, label_column, 0, max_depth)

In [ ]:
predictions = X_test.apply(lambda row: predict(decision_tree, row), axis=1)

<ipython-input-44-a74a2bfe260b>:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif sample[tree["feature_index"]] <= tree["threshold"]:


In [ ]:
custom_accuracy = accuracy_score(X_test[label_column], predictions)
print(f"Custom Decision Tree Accuracy: {custom_accuracy:.2f}")

Custom Decision Tree Accuracy: 0.90


In [ ]:
clf = DecisionTreeClassifier(max_depth=max_depth, random_state=42)
clf.fit(X_train[features], X_train[label_column])
sklearn_predictions = clf.predict(X_test[features])
sklearn_accuracy = accuracy_score(X_test[label_column], sklearn_predictions)
print(f"Scikit-Learn Decision Tree Accuracy: {sklearn_accuracy:.2f}")

Scikit-Learn Decision Tree Accuracy: 1.00
